<b>Main Question: How does Brexit affects UK's sea trade with EU</b>

Sub Question:
1. How big is the trade between UK and EU compared to UK with other regions?
2. What type of cargo is the most traded in UK?
3. How does the Brexit affect trades between EU and UK?
4. How are the major cargoes affected in the period of Brexit (2016 - 2021)?

To visualize data regarding all the sub questions, we import necessary libraries as follows

In [2]:
import pandas as pd
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

We use data frame in CSV format regarding all waterborne trades from and to the UK. The data frame consists of what type of cargoes, the origins and destination of the shipping, and cargo volume throughout a certain year. We import the data frame using pandas.

In [3]:
# import
file_path = 'DataBases/UK_Shipping.csv'
df = pd.read_csv(file_path)
df

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 199853: invalid continuation byte

In [4]:
import chardet
with open(file_path, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

We scope down the data frame from 2012 to see changes of trend from 5 years before Brexit started appearing in 2016 compare to after the Brexit happened. We also make a minor modification to clean and shorten the infomation therefore it is easier to read.

In [31]:
df = pd.read_csv(file_path, encoding='ISO-8859-1')
df = df.loc[df['Year'] > 2011]

#We shorten the coumn name
df.loc[df['Region'] == 'Asia (excluding Mediterranean countries)', 'Region'] = 'Asia'

#We decided to delete this 'unspecified' region to clean the data since it also only has small number of volume
df = df.loc[~df['Region'].str.contains('Unspecified')]

#We want to use all the inward and outward cargo data
df = df.loc[df['Direction'] == 'Both Directions']
df


C:\Users\Satrya Dewantara\AppData\Local\Temp\ipykernel_25968\1499882987.py:1: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



,Cargo Group,CargoCode,CargoName,Region,Country,Direction,Year,Tonnage (thousands),Units (thousands),TEU (thousands),Unnamed: 10,Unnamed: 11
39382,Liquid Bulk,11,Liquefied gas,All Africa,Mauritius,Both Directions,2012,28.000000,0.0,NaN,NaN,NaN
39383,Liquid Bulk,11,Liquefied gas,All European Union,Denmark,Both Directions,2012,28.000000,0.0,NaN,NaN,NaN
39384,Liquid Bulk,11,Liquefied gas,All European Union,Finland,Both Directions,2012,18.000000,0.0,NaN,NaN,NaN
39385,Liquid Bulk,11,Liquefied gas,All European Union,Latvia,Both Directions,2012,3.000000,0.0,NaN,NaN,NaN
39386,Liquid Bulk,11,Liquefied gas,Asia,South Korea,Both Directions,2012,2.000000,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
67168,Other General Cargo,99,Other general cargo & containers <20',Other Europe & Mediterranean,Egypt,Both Directions,2021,1.592205,0.0,NaN,NaN,NaN
67169,Other General Cargo,99,Other general cargo & containers <20',Other Europe & Mediterranean,Libyan Arab,Both Directions,2021,0.465926,0.0,NaN,NaN,NaN
67170,Other General Cargo,99,Other general cargo & containers <20',Other Europe & Mediterranean,Montenegro,Both Directions,2021,0.062181,0.0,NaN,NaN,NaN
67171,Other General Cargo,99,Other general cargo & containers <20',Other Europe & Mediterranean,Turkey,Both Directions,2021,90.163310,0.0,NaN,NaN,NaN


<b>All UK's Sea Trade</b>

In this first section, we want to answer the <b>Sub Question 1 and 2</b> by plotting the data regarding all the sea trades in the UK (Import, Export and Domestic). We divided the information into two charts. First one is the chart with all cargoes divided by region of origins and destinations. The second one is divided by type of cargoes.

In [33]:
#We group the trade data by region of origins and destinations
df_region = df.groupby(['Year', 'Region']).sum()
df_region = df_region.reset_index()

#Then we group the trade data by Cargo Type
df_bycargo = df.groupby(['Year', 'Cargo Group']).sum()
df_bycargo = df_bycargo.reset_index()

#We make two subplots for those two data frame groups
regions = list(df.Region.unique())
cargo_types = list(df['Cargo Group'].unique())
fig_1 = make_subplots(rows=1, cols=2, subplot_titles=['UK trades by region', 'UK trade by cargo type'])

#We use for loop to add lines for every region/cargo type to the plot
for region in regions:
    df_story_1 = df_region.loc[df_region['Region'] == region]
    x1 = df_story_1['Year']
    y1 = df_story_1['Tonnage (thousands)']
    fig_1.append_trace(go.Scatter(x=x1, y=y1, name=region,
    legendgroup= 'Group1',
    legendgrouptitle_text = 'Region'), 
    row=1, col=1)
    

for cargo_type in cargo_types:
    df_story_2 = df_bycargo.loc[df_bycargo['Cargo Group'] == cargo_type]
    x1 = df_story_2['Year']
    y1 = df_story_2['Tonnage (thousands)']
    fig_1.append_trace(go.Scatter(x=x1, y=y1, name=cargo_type,
    line=dict(dash='dashdot'),
    legendgroup= 'Group2',
    legendgrouptitle_text = 'Cargo Type'),  
    row=1, col=2)

fig_1.update_yaxes(title_text='Tonnage (thousands)', row=1, col=1)
fig_1.update_yaxes(title_text='Tonnage (thousands)', row=1, col=2)
fig_1.update_layout(title='UK All Trades', template='none')
fig_1.show()

From the left chart, it can be seen that UK's biggest trade is from and to EU with approximately twice the volume of the UK Domestic.

The right chart shows that the largest volume by cargo type traded in the UK is Liquid Bulk followed by Ro-Ro. 
- Liquid bulk is any cargoes in liquid usually carried by a tanker vessels. There are many cargoes classified as liquid bulk, such as chemical, petroleum, and liqufied gas.
- Ro-Ro or Roll On/Roll off is type of cargo which have wheels so they usually do not need heavy equipment to be loaded or unloaded to/from the vessels. Some cargoes classified as Ro-Ro are cars, construction equipment, or trucks

We can see the trade between EU and UK dominates the trade volume in the UK. Therefore we want to see how that trend is affected in the period of Brexit to answer the <b>Sub Question 3</b>. To do that, we make graphs regarding the growth rate and the the EU's market share in the UK's trade.

<b>Growth Rate</b>

At this section, we want to show the growth rate of trade between EU and UK. After that we compare that growth with the trade between UK and other regions.

In [28]:
#We filter the region to EU and group it by year
df_EU = df.loc[df['Region'] == 'All European Union']
df_EU = df_EU.groupby(['Year']).sum()
df_EU = df_EU.reset_index()

#We calculate the growth rate and assign it to a new column
df_EU['Growth Rate (%)'] = df_EU['Tonnage (thousands)'].pct_change().mul(100).round(2)
df_EU = df_EU.loc[df_EU['Year'] > 2012]

#We plot it together with the cargo volume to clearly show the trend
fig_2 = make_subplots(specs=[[{"secondary_y": True}]])
fig_2.add_trace(go.Scatter(x=df_EU['Year'], y=df_EU['Tonnage (thousands)'], name='Tonnage'), secondary_y=True)
fig_2.add_trace(go.Bar(x=df_EU['Year'], y=df_EU['Growth Rate (%)'], name='Growth Rate (%)'), secondary_y=False)
fig_2.update_yaxes(range=[180000, 210000], secondary_y=True)
fig_2.update_yaxes(range=[-6, 6], secondary_y=False)
fig_2.update_yaxes(title_text='Tonnage (thousands)', secondary_y=True)
fig_2.update_yaxes(title_text='Growth Rate (%)', secondary_y=False)
fig_2.update_layout(template='none', title = 'Trades Between UK and UE', autosize=False)
fig_2.show()

In [29]:
#again, we group the data frame regarding region and year, but this time we put the region first
df_region_2 = df.groupby(['Region', 'Year']).sum()
df_region_2 = df_region_2.reset_index()

#we exclude the asutralia due to its small volume
df_region_2 = df_region_2.loc[~df_region_2['Region'].str.contains('Australasia')]

#We calculate the growth rate and assign it to a new column
df_region_2['Growth Rate (%)'] = df_region_2['Tonnage (thousands)'].pct_change().mul(100).round(2)
df_region_2 = df_region_2.loc[df_region_2['Year'] > 2012]

#We use animated graphs to see comparison throughout the period
fig_3 = px.bar(
    df_region_2, x='Region', y='Growth Rate (%)', animation_frame='Year', color='Region',
    color_discrete_map={
                'All Africa': 'gray',
                'All America': 'gray',
                'Other Europe & Mediterranean': 'gray',
                'UK Domestic': 'gray',
                'All European Union': 'crimson',
                'Asia': 'gray'},
    template='none',
    range_y=[-30,30])
fig_3.update_layout(showlegend=False, autosize=False)
fig_3.update_layout(title='UK Trade Growth Rate')


We can see a downtrend in the period of Brexit after 2016 compared to the period before that. Moreover, after the referendum in 2018, the volume drops significantly.

In the second graph we can see the growth rate of UK-EU trade (red) has no positive growth rate after 2017, where trade between UK and other countries have some increase in certain year.

To see from another angle, we plot the graphs to show how the EU's market share in the UK is affected by the brexit.

In [34]:
#We make a new data frame to assign new column for proportion
df_region_3 = df.groupby(['Year', 'Region']).sum()
df_region_3 = df_region.reset_index()

#We calculate the market share of every region in every year then we assign it to a new column
df_region_3['Proportion (%)'] = 100 * df_region_3['Tonnage (thousands)'] / df_region_3.groupby('Year')['Tonnage (thousands)'].transform('sum')
df_region_3 = df_region_3.sort_values(['Year', 'Tonnage (thousands)'], ascending=False)

regions = list(df.Region.unique())
regions.remove('All European Union')

#We use for loop again to plot the lines
fig_4 = go.Figure()
for region in regions:
    df_story_3 = df_region_3.loc[df_region_3['Region'] == region]
    x1 = df_story_3['Year']
    y1 = df_story_3['Proportion (%)']
    fig_4.add_trace(go.Scatter(x=x1, y=y1, name=region))

#We add the EU's line later to modify the line into dash
df_story_3_EU = df_region_3.loc[df_region_3['Region'] == 'All European Union']
fig_4.add_trace(go.Scatter(x=df_story_3_EU['Year'], y=df_story_3_EU['Proportion (%)'], name='All European Union', line=dict(dash='dashdot')))

fig_4.update_yaxes(title_text='Proportion (%)')
fig_4.update_layout(title='The UK Trade Proportion', autosize=False, template='none')
fig_4.show()

From the chart, we can see some decline after 2016 and even significant decrease after 2018 of EU's market share. On the other hand, in the Brexit period the UK Domestic trade shows some increase.

Notice that in 2020 the volume of the trade between EU and UK increase, this may happen because the trade behaviour changes due to Covid 19 started in the beginning og 2020

<b>Effect on Major Cargoes</b>

We want to look deeper into the effect of brexit to various cargo types traded between UK and EU to answer the <b>Sub Question 4</b>. First, we are going to plot the volume distribution of cargo types traded between UK and EU as follows.

In [30]:
#We filter data to only between UK and EU and group it by cargo type
df_EU = df.loc[df['Region'] == 'All European Union']
df_EU = df_EU.groupby(['Year', 'Cargo Group']).sum()
df_EU = df_EU.reset_index()

#Standard plot using line chart
fig = px.line(df_EU, x='Year', y='Tonnage (thousands)', color='Cargo Group', markers='Cargo Group', template='none')
fig.update_layout(title = 'UK - EU by Cargo Group', autosize=False)
fig.show()

From the Chart, we can see the majority of cargoes traded between UK and EU are Ro-Ro and Liquid Bulk. There is a slight different between this graph and the first graph regading "UK's traded by Cargo Type". In that graph, the largest volume is Liquid bulk instead of Ro-Ro.

This chart also shows a sign of downtrend especially for Liquid Bulk and Ro-Ro in the period of Brexit all the way to 2021. The green line is the Lo-Lo (Lift Off & Lift On) or commonly known as Container. Containers are used to carry variuos of goods, and one of the most common is to carry consumer goods. Notice that the container trade between UK and EU is not really affected from 2016 until it starts declining after the Brexit Referendum in 2018, but then return to the uptrend in 2020. This answers the previous hypothesis about the increase of EU's proportion in 2020. This may happen becuase people tend to order more goods online during the Covid period therefore increase the number of containers.

<b>Conclusion</b>

1. The trade volume between UK and EU is the largest compared to UK with other regions.
2. The largest volume of cargo type traded from and to UK is Liquid Bulk and followed by Ro-Ro.
3. In the period of brexit discussion to the referendum (2016-2018), trade volume between UK and EU are generally affected
4. Ro-Ro and Liquid bulk, as the most traded cargoes between UK and EU are declining during and after the Brexit. However, the container only affected in the year of referendum (2018)

